In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime as dt

In [2]:
historical_csv_path = Path(r"./resources/historical_data.csv")
doordash_raw_df = pd.read_csv(historical_csv_path)
display(doordash_raw_df.head())
print(doordash_raw_df.isnull().count())


,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


market_id                                       197428
created_at                                      197428
actual_delivery_time                            197428
store_id                                        197428
store_primary_category                          197428
order_protocol                                  197428
total_items                                     197428
subtotal                                        197428
num_distinct_items                              197428
min_item_price                                  197428
max_item_price                                  197428
total_onshift_dashers                           197428
total_busy_dashers                              197428
total_outstanding_orders                        197428
estimated_order_place_duration                  197428
estimated_store_to_consumer_driving_duration    197428
dtype: int64


In [3]:
# check for data types
doordash_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [4]:
# Convert the two time columns to datetime
doordash_raw_df[['created_at', 'actual_delivery_time']] = doordash_raw_df[['created_at', 'actual_delivery_time']].apply(pd.to_datetime)
doordash_clean_df = doordash_raw_df.copy()

In [5]:
# creating target for regression model
doordash_clean_df['actual_total_delivery_duration'] = (doordash_clean_df['actual_delivery_time'] - doordash_clean_df['created_at']) 

In [6]:
# new busher dasher ratio
doordash_clean_df['busy_dashers_ratio'] = doordash_clean_df['total_busy_dashers'] / doordash_clean_df['total_onshift_dashers']

In [7]:
# non preperation sums
doordash_clean_df['estimated_non_prep_duration'] = doordash_clean_df['estimated_order_place_duration'] + doordash_clean_df['estimated_store_to_consumer_driving_duration']

In [8]:
# inspecting nominal data to verify if I should OneHotEncode
doordash_clean_df[['market_id','store_id','order_protocol']].nunique()

market_id            6
store_id          6743
order_protocol       7
dtype: int64

In [9]:
order_protocol_dummies = pd.get_dummies(doordash_clean_df['order_protocol'])
order_protocol_dummies = order_protocol_dummies.add_prefix('order_protocol_')
order_protocol_dummies.head(3)

,order_protocol_1.0,order_protocol_2.0,order_protocol_3.0,order_protocol_4.0,order_protocol_5.0,order_protocol_6.0,order_protocol_7.0
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0


In [10]:
market_id_dummies = pd.get_dummies(doordash_clean_df['market_id'])
market_id_dummies = market_id_dummies.add_prefix('market_id_')
market_id_dummies.head(3)

,market_id_1.0,market_id_2.0,market_id_3.0,market_id_4.0,market_id_5.0,market_id_6.0
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0


In [11]:
def fill(store_id):
    '''Return primary store category from the dictionary'''
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan

# ceate dictionary with most repeated categories of each store to fill null rows where it is possible
store_id_unique = doordash_clean_df['store_id'].unique().tolist()
store_id_and_category = {store_id: doordash_clean_df[doordash_clean_df['store_id'] == store_id] \
                        ['store_primary_category'] \
                        .mode() for store_id in store_id_unique}

doordash_clean_df['nan_free_store_primary_category'] = doordash_clean_df['store_id'].apply(fill)

In [12]:
store_primary_category_dummies = pd.get_dummies(doordash_clean_df['nan_free_store_primary_category'])
store_primary_category_dummies = store_primary_category_dummies.add_prefix('category_')
store_primary_category_dummies.head(3)

,category_afghan,category_african,category_alcohol,category_alcohol-plus-food,category_american,category_argentine,category_asian,category_barbecue,category_belgian,category_brazilian,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# cleanup original columns
original_columns = ['created_at', 'market_id', 'store_id', 'order_protocol', 
                    'store_primary_category', 'actual_delivery_time', 
                    'nan_free_store_primary_category', 'actual_total_delivery_duration']
train_df = doordash_clean_df.drop(columns=original_columns)
train_df.head()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,busy_dashers_ratio,estimated_non_prep_duration,nan_free_store_primary_category
0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,0.424242,1307.0,american
1,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,2.000000,1136.0,indian
2,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,0.000000,1136.0,indian
3,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,1.000000,735.0,indian
4,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,1.000000,1096.0,indian


In [17]:
train_df = pd.concat([train_df, order_protocol_dummies, market_id_dummies, store_primary_category_dummies], axis=1)

In [18]:
# align dtype
train_df = train_df.astype('float32')
train_df.head(3)

ValueError: could not convert string to float: 'american'

In [ ]:
train_df.describe()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
count,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,181166.000000,181166.000000,181166.000000,197428.000000,196902.000000,...,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000
mean,3.196391,2682.242188,2.670791,686.189941,1159.415405,44.808094,41.739746,58.050064,309.313446,545.358765,...,0.000815,0.000187,0.005465,0.011300,0.000740,0.037254,0.001175,0.001433,0.004234,0.030928
std,2.665833,1822.968994,1.629398,522.068542,558.358337,34.524193,32.146065,52.657368,90.211159,219.350143,...,0.028535,0.013687,0.073625,0.105705,0.027175,0.189502,0.034246,0.037816,0.064904,0.173167
min,1.000000,0.000000,1.000000,-86.000000,0.000000,-4.000000,-5.000000,-6.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1400.000000,1.000000,299.000000,800.000000,17.000000,15.000000,17.000000,251.000000,382.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,2200.000000,2.000000,595.000000,1095.000000,37.000000,34.000000,41.000000,251.000000,544.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,3395.000000,3.000000,949.000000,1395.000000,65.000000,62.000000,85.000000,446.000000,702.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,411.000000,27100.000000,20.000000,14700.000000,14700.000000,171.000000,154.000000,285.000000,2715.000000,2088.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_df['busy_dashers_ratio'].describe()

count    1.775900e+05
mean              NaN
std               NaN
min              -inf
25%      8.269231e-01
50%      9.622642e-01
75%      1.000000e+00
max               inf
Name: busy_dashers_ratio, dtype: float64

In [ ]:
# check infinity values
np.where(np.any(~np.isfinite(train_df), axis=0) == True)

(array([ 0,  6,  7,  8, 10, 11, 12], dtype=int64),)

In [ ]:
# replace inf value with nan for dropping
train_df = train_df.replace([np.inf, -np.inf], np.nan)
# drop nans
train_df =  train_df.dropna()

In [ ]:
train_df.shape

(176173, 100)